In [1]:
import pandas as pd
from scipy.stats import ttest_ind

После импорта соответствующих библиотек, с помощью pd.read_csv выгрузим нужный файл с данными. В нем мы имеем 
следующие колонки :

user_id - идентификатор клиента,

flag_group - флаг группы (контрольная=control, тестовая=pilot)

id_city - город

flag_conv - маркер совершения покупки (совершена=1, не совершена=0)


После выгрузки добавим аргумент сепаратора со значением ";"

In [2]:
df = pd.read_csv('C://Users/Petr/Desktop/Обучение/AB_test(8_lesson)/Taxi_AB_test.csv', sep=';')
df

,user_id,flag_group,id_city,flag_conv
0,2001030,pilot,Saint-Petersbourg,1
1,2004390,control,Saint-Petersbourg,0
2,2003763,pilot,Novosibirsk,0
3,2000955,pilot,Rostov,0
4,2002464,pilot,Krasnodar,0
...,...,...,...,...
5323,2001065,control,Moscow,0
5324,2005214,control,Tumen,0
5325,2005059,pilot,Kazan,0
5326,2000909,control,Saint-Petersbourg,0


Сгруппируем сумму значений "nflag_purchase" контрольной и тестовой групп по городам и 
отсортируем результаты по городам.

In [4]:
df_gr = df.groupby(['flag_group','id_city']).agg({'flag_conv':'sum',
                                            'user_id':'count'}).sort_values(by = 'id_city').reset_index()
df_gr

,flag_group,id_city,flag_conv,user_id
0,control,Kazan,65,337
1,pilot,Kazan,81,347
2,control,Krasnodar,31,138
3,pilot,Krasnodar,30,108
4,control,Moscow,150,897
5,pilot,Moscow,189,861
6,control,Novosibirsk,89,427
7,pilot,Novosibirsk,96,441
8,control,Rostov,24,123
9,pilot,Rostov,31,138


Добавим в нашу таблицу столбец с метрикой конверсии (деление суммы покупок в группе на сумму клиентов в группе)

In [5]:
df_gr['conv'] = df_gr['flag_conv']/df_gr['user_id']
df_gr

,flag_group,id_city,flag_conv,user_id,conv
0,control,Kazan,65,337,0.192878
1,pilot,Kazan,81,347,0.233429
2,control,Krasnodar,31,138,0.224638
3,pilot,Krasnodar,30,108,0.277778
4,control,Moscow,150,897,0.167224
5,pilot,Moscow,189,861,0.219512
6,control,Novosibirsk,89,427,0.208431
7,pilot,Novosibirsk,96,441,0.217687
8,control,Rostov,24,123,0.195122
9,pilot,Rostov,31,138,0.224638


Рассчитаем показатели теста Стьюдета: s (т-критерий) и p (p_value) для каждого города, чтобы понять
насколько значима разница п.п конверсий для каждого города. 

In [11]:
for i in df['id_city'].unique():
  print(i)
  s, p = ttest_ind(df[(df['id_city']==i) & (df['flag_group']=='control')]['flag_conv'], df[(df['id_city']==i) & (df['flag_group']=='pilot')]['flag_conv'])
  print(p)
  print()


Saint-Petersbourg
0.00024266025420121437

Novosibirsk
0.7395558235085458

Rostov
0.5612160660057447

Krasnodar
0.34016121489920315

Kazan
0.19620172513604184

Tumen
0.3648896448499769

Moscow
0.005452749264597743



Как мы знаем, если p_value > alpha (0.05), то разницы между выборками нет, и гипотеза Hо побеждает. 
Отберем города, где есть разница, т.е. p_value < 0.05: это две столицы, в остальных городах марктинговая акция, 
результаты А/Б тестов которой мы посчитали, не имеет финансового смысла. Мы можем делиться соответствующими 
выводами с бизнесом. 